## X-axis: Amp_steps

In [9]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as px

# Data creation
data = {
    'reset': ['active', 'active', 'active', 'thermal', 'thermal', 'thermal', 
              'active', 'active', 'active', 'active', 'active', 'active',
              'active', 'active', 'active', 'active', 'active', 'active',
              'thermal', 'active', 'active', 'active', 'active', 'active',
              'active', 'active', 'active', 'active', 'active', 'active',
              'active', 'active', 'active', 'active', 'active', 'active',
              'active', 'thermal', 'thermal'],
    'max_attempts': ['1', '1', '1', 'none', 'none', 'none', 
                     '5', '5', '5', '15', '15', '15',
                     '30', '30', '30', '1', '15', '30',
                     'none', '30', '15', '5', '1', '1',
                     '1', '1', '1', '1', '1', '5',
                     '5', '5', '5', '15', '15', '15',
                     '15', 'none', 'none'],
    'amp_steps': [53, 53, 53, 53, 53, 53, 
                  53, 53, 53, 53, 53, 53,
                  53, 53, 53, 53, 53, 53,
                  53, 53, 53, 53, 53, 5300,
                  530, 5300, 5300, 530, 530, 530,
                  530, 530, 530, 5300, 5300, 5300,
                  5300, 530, 5300],
    'shots': [100, 500, 1000, 100, 500, 1000,
              100, 500, 1000, 100, 500, 1000,
              100, 500, 1000, 1, 1, 1,
              3000, 3000, 3000, 3000, 3000, 3000,
              30, 500, 1000, 1000, 500, 100,
              500, 1000, 3000, 100, 500, 1000,
              3000, 100, 100],
    'elapse_time': [8.35, 8.69, 8.53, 16.65, 25.93, 41.7,
                    8.58, 16.69, 17.965, 8.365, 16.71, 25.295,
                    8.37, 25.08, 33.44, 8.325, 8.365, 8.355,
                    94.63, 76.25, 41.67, 24.99, 16.72, 328.38,
                    8.35, 67.06, 114.82, 24.95, 16.66, 16.64,
                    33.38, 49.92, 116.69, 47.9, 192.06, 386.65,
                    1153.2, 41.71, 320.12]
}

df = pd.DataFrame(data)

# Create legend combinations (using reset, max_attempts, and shots)
df['legend'] = df['reset'] + '_att' + df['max_attempts'].astype(str) + '_shots' + df['shots'].astype(str)

# Get unique legend values
unique_legends = df['legend'].unique()

# Create color palette
colors = px.colors.qualitative.Plotly + px.colors.qualitative.Set1 + px.colors.qualitative.Set2

# Create Plotly figure
fig = go.Figure()

# List to store regression analysis results
regression_results = []

# Add scatter plot and regression line for each legend group
for i, legend in enumerate(unique_legends):
    subset = df[df['legend'] == legend]
    color = colors[i % len(colors)]
    
    # Add scatter plot
    fig.add_trace(go.Scatter(
        x=subset['amp_steps'],
        y=subset['elapse_time'],
        mode='markers',
        name=legend,
        marker=dict(
            size=8,
            color=color
        ),
        legendgroup=legend,  # Group with same legend
        showlegend=True,
        hovertemplate='<b>%{fullData.name}</b><br>Amp Steps: %{x}<br>Elapse Time: %{y:.2f}s<extra></extra>'
    ))
    
    # Add regression line only when there are 2 or more unique amp_steps values
    if subset['amp_steps'].nunique() >= 2:
        X = subset['amp_steps'].values.reshape(-1, 1)
        y = subset['elapse_time'].values
        
        # Train linear regression model
        model = LinearRegression()
        model.fit(X, y)
        
        # Generate x values for regression line
        x_min, x_max = subset['amp_steps'].min(), subset['amp_steps'].max()
        
        # Create appropriate range for regression line
        x_range = np.linspace(x_min, x_max, 50)
        y_pred = model.predict(x_range.reshape(-1, 1))
        
        # Add regression line
        fig.add_trace(go.Scatter(
            x=x_range,
            y=y_pred,
            mode='lines',
            name=f'{legend}_regression',
            line=dict(color=color, width=2, dash='dash'),
            legendgroup=legend,  # Group with same legend
            showlegend=False  # Hide regression line from legend
        ))
        
        # Save regression analysis results
        regression_results.append({
            'legend': legend,
            'a (slope)': model.coef_[0],
            'b (intercept)': model.intercept_,
            'n_points': len(subset),
            'unique_x_values': subset['amp_steps'].nunique(),
            'equation': f'y = {model.coef_[0]:.6f}x + {model.intercept_:.2f}'
        })

# Update layout with log scale option for x-axis
fig.update_layout(
    title='Amp Steps vs Elapse Time with Individual Linear Regressions',
    xaxis_title='Amp Steps (x-axis)',
    yaxis_title='avg elapse_time (s) (y-axis)',
    hovermode='closest',
    width=1200,
    height=700,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(size=9)
    ),
    xaxis=dict(
        type='linear',  # Log scale to better visualize 53, 530, 5300
        tickvals=[53, 530, 5300],
        ticktext=['53', '530', '5300']
    )
)

# Display graph
fig.show()

# Print regression analysis results
print("\n" + "="*70)
print("Linear Regression Analysis Results for Each Legend (y = ax + b)")
print("="*70)

# Convert results to DataFrame for better display
if regression_results:
    results_df = pd.DataFrame(regression_results)
    results_df = results_df.sort_values('legend')
    
    for idx, row in results_df.iterrows():
        print(f"\nLegend: {row['legend']}")
        print(f"  Equation: {row['equation']}")
        print(f"  Slope (a): {row['a (slope)']:.8f}")
        print(f"  Intercept (b): {row['b (intercept)']:.4f}")
        print(f"  Number of data points: {row['n_points']}")
        print(f"  Unique amp_steps values: {row['unique_x_values']}")
    
    # Print summary table
    print("\n" + "="*70)
    print("Regression Analysis Summary Table")
    print("="*70)
    
    # Print formatted table
    summary_df = results_df[['legend', 'a (slope)', 'b (intercept)', 'n_points', 'unique_x_values']].copy()
    summary_df['a (slope)'] = summary_df['a (slope)'].round(8)
    summary_df['b (intercept)'] = summary_df['b (intercept)'].round(4)
    
    # Set pandas display options
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    
    print(summary_df.to_string(index=False))
else:
    print("\nNo regression analysis possible - need at least 2 different amp_steps values per legend group")

# Additional analysis: Group by specific parameters
print("\n" + "="*70)
print("Analysis by Parameter Groups (where regression is possible)")
print("="*70)

# Group by shots and max_attempts to see amp_steps effect
grouped = df.groupby(['shots', 'max_attempts'])
analysis_count = 0
for (shots, max_attempts), group in grouped:
    if group['amp_steps'].nunique() >= 2:  # Need at least 2 different x values
        X = group['amp_steps'].values.reshape(-1, 1)
        y = group['elapse_time'].values
        model = LinearRegression()
        model.fit(X, y)
        print(f"\nShots={shots}, Max_attempts={max_attempts}:")
        print(f"  Equation: y = {model.coef_[0]:.8f}x + {model.intercept_:.2f}")
        print(f"  Data points: {len(group)}")
        print(f"  Amp_steps values: {sorted(group['amp_steps'].unique())}")
        analysis_count += 1

if analysis_count == 0:
    print("\nNo groups with multiple amp_steps values found for regression analysis")

# CSV save option
save_to_csv = input("\n\nWould you like to save the results to a CSV file? (y/n): ")
if save_to_csv.lower() == 'y' and regression_results:
    results_df.to_csv('regression_results_ampsteps.csv', index=False)
    print("Results have been saved to regression_results_ampsteps.csv")
elif save_to_csv.lower() == 'y':
    print("No regression results to save")


Linear Regression Analysis Results for Each Legend (y = ax + b)

Legend: active_att15_shots100
  Equation: y = 0.007535x + 7.97
  Slope (a): 0.00753478
  Intercept (b): 7.9657
  Number of data points: 2
  Unique amp_steps values: 2

Legend: active_att15_shots1000
  Equation: y = 0.068869x + 21.64
  Slope (a): 0.06886888
  Intercept (b): 21.6449
  Number of data points: 2
  Unique amp_steps values: 2

Legend: active_att15_shots3000
  Equation: y = 0.211841x + 30.44
  Slope (a): 0.21184105
  Intercept (b): 30.4424
  Number of data points: 2
  Unique amp_steps values: 2

Legend: active_att15_shots500
  Equation: y = 0.033419x + 14.94
  Slope (a): 0.03341910
  Intercept (b): 14.9388
  Number of data points: 2
  Unique amp_steps values: 2

Legend: active_att1_shots1000
  Equation: y = 0.019683x + 10.84
  Slope (a): 0.01968298
  Intercept (b): 10.8350
  Number of data points: 3
  Unique amp_steps values: 3

Legend: active_att1_shots3000
  Equation: y = 0.059398x + 13.57
  Slope (a): 0.05939